In [ ]:
Threads.nthreads()

In [ ]:
BR = Array{Float64,3}(undef, 256, 256, 129);
BZ = Array{Float64,3}(undef, 256, 256, 129);
BPhi = Array{Float64,3}(undef, 256, 256, 129);

In [ ]:
using NetCDF
ncfile = NetCDF.open("/home/dell/fusiondata/w7x/w7x_standard.nc", readdimvar=true)

In [ ]:
using TensorCast

In [ ]:
struct CylindricalVectorField{T}
    R::Vector{T}
    Z::Vector{T}
    Phi::Vector{T}
    VR::Array{T,3}
    VZ::Array{T,3}
    VPhi::Array{T,3}
end

In [ ]:
struct CylindricalScalarField{T}
    R::Vector{T}
    Z::Vector{T}
    Phi::Vector{T}
    value::Array{T,3}
end

$\nabla\cdot\vec{B}=\partial_{R} B_{R} + \partial_{Z} B_{Z} + (B_{R}+\partial_{\phi} B_{\phi})/R$

In [ ]:
function divergence(v::CylindricalVectorField)
    dR = v.R[2]-v.R[1];
    dZ = v.Z[2]-v.Z[1];
    dPhi = v.Phi[2]-v.Phi[1];
    ans = Array{eltype(v.VR),3}(undef, length(v.R), length(v.Z), length(v.Phi) );
    ans[2:end-1,2:end-1,:] .= (v.VR[3:end,2:end-1,:].-v.VR[1:end-2,2:end-1,:])./(2*dR) .+ (v.VZ[2:end-1,3:end,:].-v.VZ[2:end-1,1:end-2,:])./(2*dZ);
    for iR in 2:length(v.R)-1
        ans[iR,:,2:end-1] .+= (v.VR[iR,:,2:end-1] .+ (v.VPhi[iR,:,3:end].-v.VPhi[iR,:,1:end-2])./(2*dPhi) ) ./ R[iR];
        ans[iR,:,1] .+= (v.VR[iR,:,1] .+ (v.VPhi[iR,:,2].-v.VPhi[iR,:,end-1])./(2*dPhi) ) ./ R[iR];
        ans[iR,:,end] .+= (v.VR[iR,:,1] .+ (v.VPhi[iR,:,2].-v.VPhi[iR,:,end-1])./(2*dPhi) ) ./ R[iR];
    end
    return CylindricalScalarField(v.R, v.Z, v.Phi, ans )
end


In [ ]:
divergence(Bfield).value[:,50,2]

In [ ]:

function magnitude(v::CylindricalVectorField)
    return .√(v.VR.^2 .+ v.VZ.^2 .+ v.VPhi.^2 )
end

In [ ]:
function cross(v1::CylindricalVectorField, v2::CylindricalVectorField)
    # TODO: check R, Z, Phi grid are identical
    return CylindricalVectorField(
        v1.R, v1.Z, v1.Phi,
        v1.VPhi*v2.VZ - v1.VZ*v2.VPhi, 
        v1.VR*v2.VPhi - v1.VPhi*v2.VR,
        v1.VZ*v2.VR   - v1.VR*v2.VZ)
end

$\vec{v}\cdot\nabla (u) = v_{R} \dfrac{\partial (u)}{\partial R} + v_{Z} \dfrac{\partial (u)}{\partial Z} +  \dfrac{v_{\phi}}{R} \dfrac{\partial (u)}{\partial \phi}$

$\vec{v}_{1}\cdot\nabla (\vec{v}_{2}) = v_{1,R} \dfrac{\partial (\vec{v}_{2})}{\partial R} + v_{1,Z} \dfrac{\partial (\vec{v}_{2})}{\partial Z} +  \dfrac{v_{1,\phi}}{R} \dfrac{\partial (\vec{v}_{2})}{\partial \phi}$

$=v_{1,R}(\dfrac{\partial v_{2,R} }{\partial R} \hat{\vec{e}}_{R} + \dfrac{\partial v_{2,Z} }{\partial R} \hat{\vec{e}}_{Z}   +\dfrac{\partial v_{2,\phi} }{\partial R} \hat{\vec{e}}_{\phi}    ) $

$+v_{1,Z}(\dfrac{\partial v_{2,R} }{\partial Z} \hat{\vec{e}}_{R} + \dfrac{\partial v_{2,Z} }{\partial Z} \hat{\vec{e}}_{Z}   +\dfrac{\partial v_{2,\phi} }{\partial Z} \hat{\vec{e}}_{\phi}    ) $

$+\dfrac{v_{1,\phi}}{R}(\dfrac{\partial v_{2,R} }{\partial \phi} \hat{\vec{e}}_{R} + \dfrac{\partial v_{2,Z} }{\partial \phi} \hat{\vec{e}}_{Z}   +\dfrac{\partial v_{2,\phi} }{\partial \phi} \hat{\vec{e}}_{\phi}    ) $

$+\dfrac{v_{1,\phi}}{R}(v_{2,R} \hat{\vec{e}}_{\phi} + \qquad \cdots\qquad   -  v_{2,\phi} \hat{\vec{e}}_{R}    ) $

$\hat{\vec{e}}_{R}= \cos\phi \hat{\vec{e}}_{x} + \sin\phi \hat{\vec{e}}_{y}$

$\hat{\vec{e}}_{\phi}= -\sin\phi \hat{\vec{e}}_{x} + \cos\phi \hat{\vec{e}}_{y}$

$\dfrac{\partial \hat{\vec{e}}_{R} }{\partial \phi} = \hat{\vec{e}}_{\phi} $

$\dfrac{\partial \hat{\vec{e}}_{\phi} }{\partial \phi} = -\hat{\vec{e}}_{R} $

In [ ]:
function directional_derivative_along_v_of_s(v::CylindricalVectorField, s::CylindricalScalarField)
    R, Z, Phi = v1.R, v1.Z, v1.Phi
    
    dR = R[2]-R[1]
    dZ = Z[2]-Z[1]
    dPhi = Phi[2]-Phi[1]
    scal = s.value
    ans = Array{eltype(v.VR),3}(undef, length(v.R), length(v.Z), length(v.Phi) );
    ans[2:end-1,2:end-1,:] .= v.VR[2:end-1,2:end-1,:] .* ( (scal[3:end,2:end-1,:].-scal[1:end-2,2:end-1,:])./(2*dR) );
    ans[2:end-1,2:end-1,:].+= v.VZ[2:end-1,2:end-1,:] .* ( (scal[2:end-1,3:end,:].-scal[2:end-1,1:end-2,:])./(2*dZ) );
    for iR in 2:length(v.R)-1
        ans[iR,:,2:end-1] .+=  v.VPhi[iR,:,2:end-1]./R[iR] .* (scal[iR,:,3:end].-scal[iR,:,1:end-2])./(2*dPhi)  ;
        ans[iR,:,1] .+= v.VPhi[iR,:,1]./R[iR] .* (scal[iR,:,2].-scal[iR,:,end-1])./(2*dPhi) ;
        ans[iR,:,end] .+= v.VPhi[iR,:,1]./R[iR]  .* (scal[iR,:,2].-scal[iR,:,end-1])./(2*dPhi) ;
    end
    return CylindricalScalarField(R, Z, Phi, ans)
end

function directional_derivative_along_v1_of_v2(v1::CylindricalVectorField, v2::CylindricalVectorField)
    R, Z, Phi = v1.R, v1.Z, v1.Phi
    
    dR = R[2]-R[1]
    dZ = Z[2]-Z[1]
    dPhi = Phi[2]-Phi[1]
    ans_VR = Array{eltype(v1.VR),3}(undef, length(R), length(Z), length(Phi) );
    ans_VZ = Array{eltype(v1.VR),3}(undef, length(R), length(Z), length(Phi) );
    ans_VPhi = Array{eltype(v1.VR),3}(undef, length(R), length(Z), length(Phi) );
    ans_VR[2:end-1,2:end-1,:] .= v1.VR[2:end-1,2:end-1,:] .* ( (v2.VR[3:end,2:end-1,:].-v2.VR[1:end-2,2:end-1,:])./(2*dR) );
    ans_VZ[2:end-1,2:end-1,:] .= v1.VR[2:end-1,2:end-1,:] .* ( (v2.VZ[3:end,2:end-1,:].-v2.VZ[1:end-2,2:end-1,:])./(2*dR) );
    ans_VPhi[2:end-1,2:end-1,:] .= v1.VR[2:end-1,2:end-1,:] .* ( (v2.VPhi[3:end,2:end-1,:].-v2.VPhi[1:end-2,2:end-1,:])./(2*dR) );
    
    ans_VR[2:end-1,2:end-1,:] .+= v1.VZ[2:end-1,2:end-1,:] .* ( (v2.VR[2:end-1,3:end,:].-v2.VR[2:end-1,1:end-2,:])./(2*dZ) );
    ans_VZ[2:end-1,2:end-1,:] .+= v1.VZ[2:end-1,2:end-1,:] .* ( (v2.VZ[2:end-1,3:end,:].-v2.VZ[2:end-1,1:end-2,:])./(2*dZ) );
    ans_VPhi[2:end-1,2:end-1,:] .+= v1.VZ[2:end-1,2:end-1,:] .* ( (v2.VPhi[2:end-1,3:end,:].-v2.VPhi[2:end-1,1:end-2,:])./(2*dZ) );
    
    for iR in 2:length(R)-1
        # For the 0 < \phi < 2\pi/n sections
        ans_VR[iR,:,2:end-1] .+= v1.VPhi[iR,:,2:end-1]./R[iR] .* ( (v2.VR[iR,:,3:end].-v2.VR[iR,:,1:end-2])./(2*dPhi) ) ;
        ans_VZ[iR,:,2:end-1] .+= v1.VPhi[iR,:,2:end-1]./R[iR] .* ( (v2.VZ[iR,:,3:end].-v2.VZ[iR,:,1:end-2])./(2*dPhi) );
        ans_VPhi[iR,:,2:end-1] .+= v1.VPhi[iR,:,2:end-1]./R[iR] .* ( (v2.VPhi[iR,:,3:end].-v2.VPhi[iR,:,1:end-2])./(2*dPhi) );
        
        ans_VPhi[iR,:,2:end-1] .+= v1.VPhi[iR,:,2:end-1]./R[iR] .* v2.VR[iR,:,2:end-1];
        ans_VR[iR,:,2:end-1] .-= v1.VPhi[iR,:,2:end-1]./R[iR] .* v2.VPhi[iR,:,2:end-1];
        
        # For the \phi=0 section
        ans_VR[iR,:,1] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VR[iR,:,2].-v2.VR[iR,:,end-1])./(2*dPhi) );
        ans_VZ[iR,:,1] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VZ[iR,:,2].-v2.VZ[iR,:,end-1])./(2*dPhi) );
        ans_VPhi[iR,:,1] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VPhi[iR,:,2].-v2.VPhi[iR,:,end-1])./(2*dPhi) );
        
        ans_VPhi[iR,:,1] .+= v1.VPhi[iR,:,1]./R[iR] .* v2.VR[iR,:,1];
        ans_VR[iR,:,1] .-= v1.VPhi[iR,:,1]./R[iR] .* v2.VPhi[iR,:,1];
        
        # For the \phi=2\pi/n section
        ans_VR[iR,:,end] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VR[iR,:,2].-v2.VR[iR,:,end-1])./(2*dPhi) ) ;
        ans_VZ[iR,:,end] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VZ[iR,:,2].-v2.VZ[iR,:,end-1])./(2*dPhi) );
        ans_VPhi[iR,:,end] .+= v1.VPhi[iR,:,1]./R[iR] .* ( (v2.VPhi[iR,:,2].-v2.VPhi[iR,:,end-1])./(2*dPhi) );
        
        ans_VPhi[iR,:,end] .+= v1.VPhi[iR,:,1]./R[iR] .* v2.VR[iR,:,1];
        ans_VR[iR,:,end] .-= v1.VPhi[iR,:,1]./R[iR] .* v2.VPhi[iR,:,1];
    end
    
    return CylindricalVectorField(R,Z,Phi, ans_VR, ans_VZ, ans_VPhi)
end

In [ ]:
Bfield = CylindricalVectorField(R,Z,Phi,BR,BZ,BPhi);

In [ ]:
size( divergence(Bfield).value ) 

In [ ]:
rminb = NetCDF.ncread("/home/dell/fusiondata/w7x/w7x_standard.nc", "rminb")[1];
rmaxb = NetCDF.ncread("/home/dell/fusiondata/w7x/w7x_standard.nc", "rmaxb")[1];
zminb = NetCDF.ncread("/home/dell/fusiondata/w7x/w7x_standard.nc", "zminb")[1];
zmaxb = NetCDF.ncread("/home/dell/fusiondata/w7x/w7x_standard.nc", "zmaxb")[1];

In [ ]:
using Ranges
R = collect( range(rminb, stop=rmaxb, length=256) );
Z = collect( range(zminb, stop=zmaxb, length=256) );
Phi = collect( range(0.0, stop=2pi/5, length=129) );

In [ ]:
for iPhi = 1:length(Phi)-1
    BR[:,:,iPhi] = ncfile.vars["B_R"][:,:,iPhi];
    BZ[:,:,iPhi] = ncfile.vars["B_Z"][:,:,iPhi];
    BPhi[:,:,iPhi] = ncfile.vars["B_phi"][:,:,iPhi];
end
BR[:,:,end] = ncfile.vars["B_R"][:,:,1];
BZ[:,:,end] = ncfile.vars["B_Z"][:,:,1];
BPhi[:,:,end] = ncfile.vars["B_phi"][:,:,1];

In [ ]:
dirderi_alongB_B = directional_derivative_along_v1_of_v2(Bfield, Bfield);


In [ ]:
heatmap_test = transpose( range(1:50)^3 ) .+ range(1:4);

In [ ]:

heatmap((1:50), 1:4, transpose(heatmap_test), clims=(0, 2))

$$\begin{bmatrix}
B_{R}\\ 
B_{\phi}\\ 
B_{Z}
\end{bmatrix} = \begin{bmatrix}
\cos\phi & \sin\phi & \\ 
-\sin\phi & \cos\phi & \\ 
 &  & 1
\end{bmatrix} \begin{bmatrix}
B_{x}\\ 
B_{y}\\ 
B_{z}
\end{bmatrix}$$

$$\begin{bmatrix}
B_{x}\\ 
B_{y}\\ 
B_{z}
\end{bmatrix} = \begin{bmatrix}
\cos\phi & -\sin\phi & \\ 
\sin\phi & \cos\phi & \\ 
 &  & 1
\end{bmatrix} \begin{bmatrix}
B_{R}\\ 
B_{\phi}\\ 
B_{Z}
\end{bmatrix}$$

In [ ]:

Bx = similar(BR)
By = similar(BR)
Bz = BZ
for iPhi = 1:129
    phi = Phi[iPhi]
    Bx[:,:,iPhi] = BR[:,:,iPhi] .* cos(phi) - BPhi[:,:,iPhi] .* sin(phi);
    By[:,:,iPhi] = BR[:,:,iPhi] .* sin(phi) + BPhi[:,:,iPhi] .* cos(phi);
end

In [ ]:
import DifferentialEquations as DE

In [ ]:
# using GridInterpolations
using Interpolations

In [ ]:

Bx_interp = linear_interpolation((R,Z,Phi), Bx);
By_interp = linear_interpolation((R,Z,Phi), By);
Bz_interp = linear_interpolation((R,Z,Phi), Bz);

In [ ]:
Bz_interp(x0[1], x0[2], x0[3])

In [ ]:
newaxis = [CartesianIndex()]  

In [ ]:

BR_interp = linear_interpolation((R,Z,Phi), BR);
BZ_interp = linear_interpolation((R,Z,Phi), BZ);
BPhi_interp = linear_interpolation((R,Z,Phi), BPhi);

In [ ]:
using TensorCast
using Memoization

In [ ]:
@cast RBRoBPhi[iR,iZ,iPhi] := R[iR] * BR[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];
@cast RBZoBPhi[iR,iZ,iPhi] := R[iR] * BZ[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];

In [ ]:
size(RBRoBPhi)

In [ ]:
dR = R[2] - R[1]
dZ = Z[2] - Z[1]
@memoize function partial_derivative_pRpZ(field, Rord, Zord)
    if Rord==0 && Zord==0
        return field
    elseif Rord > 0
        lastord_field = partial_derivative_pRpZ(field, Rord-1, Zord)
        return (lastord_field[3:end,:,:]- lastord_field[1:end-2,:,:]) / (2dR)
    elseif Zord > 0 
        lastord_field = partial_derivative_pRpZ(field, Rord, Zord-1)
        return (lastord_field[:,3:end,:]- lastord_field[:,1:end-2,:]) / (2dZ)
    end
end

        

In [ ]:
@memoize function partial_derivative_pRpZ_interp(field, Rord, Zord)
    return linear_interpolation( 
            (R[1+Rord:end-Rord], 
            Z[1+Zord:end-Zord], 
            Phi), 
        partial_derivative_pRpZ(field, Rord, Zord) )
end

        

In [ ]:
@time partial_derivative_pRpZ(RBRoBPhi, 7,18);

In [ ]:
A11 = partial_derivative_pRpZ_interp(RBRoBPhi,1,0);
A12 = partial_derivative_pRpZ_interp(RBRoBPhi,0,1);
A21 = partial_derivative_pRpZ_interp(RBZoBPhi,1,0); 
A22 = partial_derivative_pRpZ_interp(RBZoBPhi,0,1);


In [ ]:
FLT_A = nothing
gc()

In [ ]:
# A11(R[10],Z[10],Phi[10]), A12(R[10],Z[10],Phi[10])
@inline FLT_A(r,z,phi) = [A11(r,z,phi)  A12(r,z,phi); A21(r,z,phi)  A22(r,z,phi)]

In [ ]:
FLT_A(R[10], Z[40], Phi[120])

In [ ]:
R[iR] * BR[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi]

In [ ]:
function FLT_cartesian!(dx,x,p,t)
    r = sqrt(x[1]^2 + x[2]^2)
    z = x[3]
    phi = atan(x[2], x[1])
    phimod = mod( phi, 2pi/5 )
    
#     dx[1] = Bx_interp(r, z, phi)
#     dx[2] = By_interp(r, z, phi)
#     dx[3] = Bz_interp(r, z, phi)
    BR_ = BR_interp(r,z,phimod)
    BZ_ = BZ_interp(r,z,phimod)
    BPhi_ = BPhi_interp(r,z,phimod)
    dx[1] = BR_ * cos(phi) - BPhi_ * sin(phi)
    dx[2] = BR_ * sin(phi) + BPhi_ * cos(phi)
    dx[3] = BZ_
end

In [ ]:
function FLT_cylindrical!(dx,x,p,phi)
    phimod = mod( phi, 2pi/5 )
    dx[1] = x[1] * BR_interp(x[1], x[2], phimod) / BPhi_interp(x[1], x[2], phimod)
    dx[2] = x[1] * BZ_interp(x[1], x[2], phimod) / BPhi_interp(x[1], x[2], phimod)
end


In [ ]:
condition(x,t,integrator) = x[1]>R[end-3] || x[1] < R[3] || x[2] >Z[end-3] || x[2] < Z[3]
affect!(integrator) = DE.terminate!(integrator)
cb = DE.DiscreteCallback(condition,affect!)
roundN = 500.0
trajN = 40
# x0s = Array{Float64,2}(undef, trajN, 3);
# x0s[:,1] = range(6.0, stop=6.0, length=trajN);
# x0s[:,2].= 0.0;
# x0s[:,3] = range(-0.8, stop=0.8, length=trajN);

x0s = Array{Float64,2}(undef, trajN, 2);
x0s[:,1] = range(6.2, stop=5.4, length=trajN);
x0s[:,2] = range(-0.6, stop=1.0, length=trajN);

tspan = (0.0, roundN*2pi)
# prob = DE.ODEProblem(FLT_cartesian!, x0s[1,:], tspan)
prob = DE.ODEProblem(FLT_cylindrical!, x0s[1,:], tspan)

function prob_func(prob, i, repeat)
    DE.remake(prob, u0=x0s[i,:])
end
ensemble_prob = DE.EnsembleProblem(prob, prob_func=prob_func)
# sol = solve(prob, RK4(), abstol=1e-9, reltol=1e-9, maxiters=1e9, dt=pi/128,) #dt=pi/128, dtmax=pi/128
ensemble_sol = DE.solve(
    ensemble_prob, DE.RK4(), DE.EnsembleThreads(), trajectories=trajN, 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb )

In [ ]:
maximum(ensemble_sol[1].t)

In [ ]:

function FLT_cylindrical_Jac!(dDXpol,DXpol,p,phi)
    phimod = mod( phi, 2pi/5 )
    traj_i = p[1]
    r,z = ensemble_sol[traj_i](phi)
    dDXpol[:,:] = FLT_A(r,z,phimod) * DXpol
end

In [ ]:

prob_Jac = DE.ODEProblem(FLT_cylindrical_Jac!, [1 0; 0 1], tspan, [0])
function prob_func_Jac(prob, i, repeat)
    DE.remake(prob, tspan=[0.0, maximum(ensemble_sol[i].t) ], p=[i] )
end
ensemble_prob_Jac = DE.EnsembleProblem(prob_Jac, prob_func=prob_func_Jac)
# sol = solve(prob, RK4(), abstol=1e-9, reltol=1e-9, maxiters=1e9, dt=pi/128,) #dt=pi/128, dtmax=pi/128
ensemble_sol_Jac = DE.solve(
    ensemble_prob_Jac, DE.RK4(), DE.EnsembleThreads(), trajectories=trajN, 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, )

In [ ]:
size( reduce(vcat, [LA.eigvals(DXpol)' for DXpol in ensemble_sol_Jac[1].u] ) )

In [ ]:
import LinearAlgebra as LA
using Plots

for traj_i = 1:1
    DXpol_Phi = ensemble_sol_Jac[traj_i].t[1:10:end]
    DXpol_eigvals = reduce(vcat, [LA.eigvals(DXpol)' for DXpol in ensemble_sol_Jac[traj_i].u[1:10:end] ] )
    plot(DXpol_Phi, DXpol_eigvals)
end

In [ ]:
ensemble_sol[5](51.5pi)
# FLT_A(R[123],Z[123],Phi[23])

In [ ]:
helA = rand(9)

In [ ]:
helA[1:2:]

In [ ]:
traj_i = 10
maxphi_i = 600001
dphi_ind = 20
println("max phi:", ensemble_sol_Jac[traj_i].t[maxphi_i])
DXpol_Phi = ensemble_sol_Jac[traj_i].t[1:dphi_ind:maxphi_i]
DXpol_eigvals = reduce(vcat, [LA.eigvals(DXpol)' for DXpol in ensemble_sol_Jac[traj_i].u[1:dphi_ind:maxphi_i] ] )
plot(DXpol_Phi, abs.(DXpol_eigvals) )

In [ ]:
import LinearAlgebra as LA
using Plots
traj_i = 12
roundN_todraw = 500

DXpol_eigvals = reduce(vcat, [LA.eigvals(ensemble_sol_Jac[traj_i](2pi*i))'  for i in range(0,roundN_todraw)] )
scatter( range(0,roundN_todraw), abs.(DXpol_eigvals[:,1]), labels="abs(λ1)")
scatter!( range(0,roundN_todraw), abs.(DXpol_eigvals[:,2]), labels="abs(λ2)" )

    

In [ ]:
using Plots
# plot(sol)
plot(ensemble_sol)
# plot(ensemble_sol[1])

In [ ]:
[ensemble_sol[1](2pi*i) for i in range(0, roundN+1)]

In [ ]:
hcat( reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, 20)] )', [2pi*i for i in range(0, 20) ] )  

In [ ]:
using NPZ
orb_dict = Dict{String, AbstractArray}()
for traj_i in range(1, trajN)
#     scatRZPhi = hcat( reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )', [2pi*i for i in range(0, roundN) ] )
#     println(size(scatRZPhi))
    orb_dict[string(traj_i)] = hcat( reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )', [2pi*i for i in range(0, roundN) ] )
#     Plots.scatter!(scatx, scaty)
end

npzwrite("w7x_standard_phi_0_Poincare_orbits.npz", orb_dict )

In [ ]:
using CSV, DataFrames

for traj_i in range(1, trajN)
    orb_filename = "/home/dell/fusiondata/w7x/w7x_standard_orbits/w7x_standard_" * string(traj_i) * ".csv"
    touch(orb_filename)
    file_handle = open(orb_filename, "w")
    scatRZ = reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )
    scatR, scatZ = scatRZ[1,:], scatRZ[2,:]
    CSV.write(orb_filename, DataFrame(R = scatR, Z = scatZ, Phi = [2pi*i for i in range(0, roundN)] ) )
end




In [ ]:
using CSV
CSV.write("output.csv", DataFrame(ensemble_sol), bufsize=Int64(1e10) ) 

In [ ]:
npzwritearray(npzfile_orb,  hcat( reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, 20)] )', [2pi*i for i in range(0, 20) ] )  )

In [ ]:
import Plots
scatxy = reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, roundN)] )
scatx, scaty = scatxy[1,:], scatxy[2,:]
Plots.scatter(scatx, scaty,)
for traj_i in range(2, trajN-1)
    scatxy = reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )
    scatx, scaty = scatxy[1,:], scatxy[2,:]
    Plots.scatter!(scatx, scaty)
end


In [ ]:
using Plots
# a = rand(5,5)
# xlabel = string.(collect('A':'E'))
# ylabel = string.(collect('a':'e'))
heatmap(R, Z,  BPhi[:,:,1], aspect_ratio=:equal)

In [ ]:

heatmap(R, Z, transpose(BR[:,:,1]), aspect_ratio=:equal)

In [ ]:
heatmat = transpose(BZ[:,:,1] )
heatmap(R, Z, heatmat, aspect_ratio=:equal, c=cgrad(:balance, [ (0.0-minimum(heatmat) )/( maximum(heatmat)-minimum(heatmat) ),]))

In [ ]:
import Plots
heatmat = transpose(BPhi[:,:,80] )
Plots.heatmap(R, Z, heatmat, aspect_ratio=:equal, c=Plots.cgrad(:balance, [ (0.0-minimum(heatmat) )/( maximum(heatmat)-minimum(heatmat) ),]))

In [ ]:
scatxy = reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, ttrajN+1)] )
scatx, scaty = scatxy[1,:], scatxy[2,:]

In [ ]:
for i in range(0, 5)
    println(i)
end

In [ ]:
reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(5)] )[1,:]

In [ ]:
import Plots

Plots.scatter()